In [1]:
%pip uninstall -y tensorflow tensorflow-macos mediapipe protobuf opencv-python keras tensorboard numpy

Found existing installation: tensorflow-macos 2.15.0
Uninstalling tensorflow-macos-2.15.0:
  Successfully uninstalled tensorflow-macos-2.15.0
Found existing installation: mediapipe 0.10.9
Uninstalling mediapipe-0.10.9:
  Successfully uninstalled mediapipe-0.10.9
Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
Found existing installation: opencv-python 4.8.0.74
Uninstalling opencv-python-4.8.0.74:
  Successfully uninstalled opencv-python-4.8.0.74
Found existing installation: keras 2.15.0
Uninstalling keras-2.15.0:
  Successfully uninstalled keras-2.15.0
Found existing installation: tensorboard 2.15.2
Uninstalling tensorboard-2.15.2:
  Successfully uninstalled tensorboard-2.15.2
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install "numpy==1.26.4" --force-reinstall --no-deps --ignore-installed

  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install "tensorflow-macos==2.15.0" "mediapipe==0.10.9" "protobuf==3.20.3" "opencv-python==4.8.0.74" "keras<3.0"

  Using cached tensorflow_macos-2.15.0-cp310-cp310-macosx_12_0_arm64.whl.metadata (4.2 kB)
  Using cached mediapipe-0.10.9-cp310-cp310-macosx_11_0_universal2.whl.metadata (9.6 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached opencv_python-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
Using cached tensorflow_macos-2.15.0-cp310-cp310-macosx_12_0_arm64.whl (208.8 MB)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
Using cached mediapipe-0.10.9-cp310-cp310-macosx_11_0_universal2.whl (47.1 MB)
Using cached opencv_python-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl (33.1 MB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [tensorflow-macos][tensorflow-macos]
Note: you may need to restart the k

In [4]:
%pip uninstall -y jax jaxlib

Found existing installation: jax 0.4.13
Uninstalling jax-0.4.13:
  Successfully uninstalled jax-0.4.13
Found existing installation: jaxlib 0.4.13
Uninstalling jaxlib-0.4.13:
  Successfully uninstalled jaxlib-0.4.13
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install "jax==0.4.13" "jaxlib==0.4.13" "ml_dtypes==0.2.0"

  Using cached jax-0.4.13-py3-none-any.whl
  Using cached jaxlib-0.4.13-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
Using cached jaxlib-0.4.13-cp310-cp310-macosx_11_0_arm64.whl (60.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [jax]1/2 [jax]
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
from pathlib import Path
import math

In [7]:
# getting data from
PROJECT_ROOT = Path("/Users/sophiekim/Desktop/msds/ds 6021/ML1-Final-Project")
IMAGES_DIR = PROJECT_ROOT / "faces"
CSV_FILE = PROJECT_ROOT / "Headshots for ML(Sheet1).csv"
VISUAL_DIR = PROJECT_ROOT / "visualized_faces"
VISUAL_DIR.mkdir(exist_ok=True)
PHI = 1.618

# loading data
print("Loading metadata...")
name_to_metadata = {}

if CSV_FILE.exists():
    meta_df = pd.read_csv(CSV_FILE)
    # rename cols to standardized lowercase keys
    meta_df = meta_df.rename(columns={
        "Email": "email",
        "Name": "name",
        "race?": "race",
        "gender?": "gender"
    })

    for _, row in meta_df.iterrows():
        raw_name = str(row["name"]).strip()
        if raw_name and raw_name.lower() != "nan":
            name_to_metadata[raw_name] = {
                "name": raw_name,
                "email": str(row["email"]).strip() if pd.notna(row["email"]) else "",
                "race": str(row["race"]).strip() if pd.notna(row["race"]) else "Unknown",
                "gender": str(row["gender"]).strip() if pd.notna(row["gender"]) else "Unknown"
            }
    print(f"Loaded {len(name_to_metadata)} people from CSV")
else:
    print("Original CSV file not found")

def get_metadata(filename):
    stem = Path(filename).stem.lower()
    
    # see if it can get the person's name from the file name
    for real_name, data in name_to_metadata.items():
        if real_name.lower() in stem:
            return data
            
    # default if not found
    return {"name": "Unknown", "email": "", "race": "", "gender": ""}

# mediapipe AI face mesh 
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.01, # low confidence so it can get everyone's face
    min_tracking_confidence=0.01
)

def get_landmarks(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)
    if results.multi_face_landmarks:
        return results.multi_face_landmarks[0].landmark
    return None

def calculate_features(landmarks, w, h):
    def get_pt(idx): return np.array([landmarks[idx].x * w, landmarks[idx].y * h])

    # pixel distance
    face_height = np.linalg.norm(get_pt(10) - get_pt(152))
    face_width = np.linalg.norm(get_pt(234) - get_pt(454))
    pupil_dist = np.linalg.norm(get_pt(468) - get_pt(473))
    avg_eye_w = (np.linalg.norm(get_pt(33)-get_pt(133)) + np.linalg.norm(get_pt(263)-get_pt(362))) / 2
    nose_width = np.linalg.norm(get_pt(102) - get_pt(292))
    mouth_width = np.linalg.norm(get_pt(61) - get_pt(291))

    if avg_eye_w == 0 or nose_width == 0: return None


    face_ratio = face_height / face_width
    mouth_nose_ratio = mouth_width / nose_width
    eye_ratio = pupil_dist / avg_eye_w
    
    # calculate how close it is to 1.618
    errors = [abs(face_ratio - PHI)/PHI, abs(mouth_nose_ratio - PHI)/PHI]
    score = (1 - (sum(errors) / len(errors))) * 100

    return {
        "face_ratio": face_ratio,
        "mouth_nose_ratio": mouth_nose_ratio,
        "eye_ratio": eye_ratio,
        "golden_score": score,
        "face_height": face_height,
        "face_width": face_width,
        "nose_width": nose_width,
        "mouth_width": mouth_width
    }

def draw_visuals(img, landmarks, feats, metadata):
    img_disp = img.copy()
    h, w, _ = img.shape
    def pt(idx): return (int(landmarks[idx].x * w), int(landmarks[idx].y * h))

    # lines
    cv2.line(img_disp, pt(10), pt(152), (255, 0, 0), 2)
    cv2.line(img_disp, pt(234), pt(454), (255, 0, 0), 2)
    cv2.line(img_disp, pt(102), pt(292), (0, 255, 255), 2)
    cv2.line(img_disp, pt(61), pt(291), (255, 0, 255), 2)

    # info box on pic
    cv2.rectangle(img_disp, (0, 0), (450, 190), (0,0,0), -1)
    def txt(t, y, c=(255,255,255)): 
        cv2.putText(img_disp, t, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c, 2)

    txt(f"Name: {metadata['name']}", 30, (0, 255, 255))
    txt(f"Email: {metadata['email']}", 60, (200, 200, 200))  # <--- Shows Email on Image
    txt(f"Score: {feats['golden_score']:.1f}%", 90, (0, 255, 0))
    txt(f"Face Ratio: {feats['face_ratio']:.2f}", 120)
    txt(f"Mouth/Nose: {feats['mouth_nose_ratio']:.2f}", 150)

    return img_disp

# go!
rows = []
all_files = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.webp']:
    all_files.extend(IMAGES_DIR.rglob(ext))
    all_files.extend(IMAGES_DIR.rglob(ext.upper()))
image_paths = sorted(list(set(all_files)))

print(f"🚀 Processing {len(image_paths)} images...")

for img_path in image_paths:
    if img_path.name.startswith('.'): continue
    
    # getting people name and stuff 
    meta = get_metadata(img_path.name)
    
    row = {
        "filename": img_path.name,
        "name": meta['name'],
        "email": meta['email'],   
        "race": meta['race'],
        "gender": meta['gender'],
        "status": "No Face Detected",
        "golden_score": None
    }

    # load image
    try:
        img = cv2.imread(str(img_path))
        if img is None: raise ValueError("Load failed")
    except:
        row["status"] = "Corrupt File"
        rows.append(row)
        continue

    h, w, _ = img.shape
    landmarks = get_landmarks(img)

    if landmarks:
        feats = calculate_features(landmarks, w, h)
        if feats:
            row.update(feats)
            row["status"] = "Success"
            
            # Create Visualized Image
            vis_img = draw_visuals(img, landmarks, feats, meta)
            save_vis_path = VISUAL_DIR / f"marked_{img_path.name}"
            cv2.imwrite(str(save_vis_path), vis_img)
            
            print(f"✅ {meta['name']} - Score: {feats['golden_score']:.1f}%")
        else:
            print(f"⚠️ Math Error: {img_path.name}")
    else:
        cv2.putText(img, "NO FACE DETECTED", (50, h//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 4)
        cv2.imwrite(str(VISUAL_DIR / f"failed_{img_path.name}"), img)
        print(f"❌ No Face: {img_path.name}")

    rows.append(row)

# save df 
df = pd.DataFrame(rows)
output_path = PROJECT_ROOT / "final_data_with_demographics.csv"
df.to_csv(output_path, index=False)

print(f"\n🎉 Done!")
print(f"Visualizations saved to: {VISUAL_DIR}")
print(f"Data saved to: {output_path}")

Loading metadata...
Loaded 51 people from CSV
🚀 Processing 54 images...
✅ Sophie Kim - Score: 72.3%
✅ Unknown - Score: 65.4%


I0000 00:00:1765480263.897495       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


✅ Sabine Segaloff - Score: 70.7%
✅ Sophie Kim - Score: 69.1%
✅ Anna Li - Score: 72.3%
✅ Aniyah McWilliams - Score: 69.9%
✅ Erin Siedlecki - Score: 70.8%
✅ Ethan Meidinger - Score: 72.9%
✅ Marissa Burton - Score: 70.5%
✅ Grace George - Score: 69.2%
✅ Randa Ampah - Score: 77.2%
✅ Isabella Lu - Score: 71.3%
✅ Sophie Kim - Score: 74.4%
✅ Unknown - Score: 66.7%
✅ Thomas Blalock - Score: 70.3%
✅ Mauricio Torres - Score: 74.0%
✅ Jack Joy - Score: 69.7%
✅ Sophie Kim - Score: 69.8%
✅ Natalie Seah - Score: 73.8%
✅ Andrew Pavlak - Score: 71.0%
✅ Mason Earp - Score: 74.0%
✅ Mohini Gupta - Score: 72.7%
✅ Timothy Yeh - Score: 70.0%
✅ Harry Millspaugh - Score: 65.8%
✅ Sheyi Faparusi - Score: 67.1%
✅ Caroline Kranefuss - Score: 75.3%
✅ Terry Luedtke - Score: 65.0%
✅ Aiden Rocha - Score: 65.1%
✅ Tianyin Mao - Score: 65.9%
✅ Lino De Ros - Score: 75.9%
✅ Brooke Lumpkin - Score: 73.2%
✅ Reed Baumgardner - Score: 70.7%
✅ Jackson Kennedy - Score: 67.2%
✅ Edward Anderson - Score: 72.4%
✅ Maggie Crowner - Sco